# 导入相关包

In [2]:
# 导入相关包
import os
import pathlib as pl
import pandas as pd
import numpy as np
import re
from io import StringIO
from datetime import datetime,timedelta
import time
from IPython.core.interactiveshell import InteractiveShell
from tqdm.autonotebook import *
import pdfplumber
tqdm.pandas()
InteractiveShell.ast_node_interactivity = "all"
sys.path.append("..")

from tgrocery import Grocery


# PDF解析原始数据 
## 加载数据并采用pdfplumber抽取PDF中的文字和表格


In [3]:
# 数据准备(train_output文件中格式有点问题，需要提前用excel或者wps打开然后另存为excel文件)
train_outputs = pd.read_excel('../datasets/train_output.xlsx')

# 获取pdf中文字和表格
def extract_pdf_content(pdf_path):
    text_list = []
    table_list = []
    with pdfplumber.open(pdf_path) as pdf:
        for index_page in np.arange(0, len(pdf.pages), 1):
            # 读取多页
            page = pdf.pages[index_page]   # 第n页的信息
            text = page.extract_text()
            text_list.append(text)
            table = page.extract_tables()
            for t in table:
                table_list.append(t)
    return text_list, table_list

def get_dir_file(path):
    '''
    输入文件夹位置，输出整理好的dataframe
    '''
    path_list = os.listdir(path)
    id_list = []
    file_path_list = []
    text_list = []
    table_list = []
    for i in tqdm(path_list):
        if '.PDF' in i:
            file_path = path + i
            id_list.append(int(i.split('.')[0]))
            file_path_list.append(file_path)
            try:
                text_temp, table_temp = extract_pdf_content(file_path)
            except Exception:
                print('此pdf无法读取')
                text_temp, table_temp = [], []
            text_list.append(text_temp)
            table_list.append(table_temp)
            
    df = pd.DataFrame()
    df['sample_id'] = id_list
    df['file_path'] = file_path_list
    df['text'] = text_list
    df['tabel'] = table_list
    df = df.sort_values('sample_id')
    return df

# 文件处理太慢，可持续化保存文件
train_path = '../datasets/train.csv'
if os.path.exists(train_path):
    train_df = pd.read_csv(train_path)
else:
    train_df = get_dir_file('datasets/train_data/')
    train_df.to_csv(train_path,index=False)
    train_df = pd.read_csv(train_path)

test_path =  '../datasets/test.csv'
if os.path.exists(test_path):
    test_df = pd.read_csv(test_path)
else:
    test_df = get_dir_file('datasets/test_data/')
    test_df.to_csv(test_path,index=False)
    test_df = pd.read_csv(test_path)

train_outputs.head(2)
train_df.head(2)
test_df.head(2)

,sample_id,认购日期,理财产品名称,产品发行方名称,理财类型,认购金额(万元),产品起息日,产品到息日,产品期限,资金来源,实际购买公司名称,实际购买公司和上市公司关系,买卖方是否有关联关系,公告日期
0,1,2019-03-27,汇聚金1号,中融国际信托有限公司,信托,10000.0,2019-03-27,2019-09-23,180天,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25
1,1,2019-03-27,招商银行步步生金8699,招商银行,银行理财产品,200.0,2019-03-27,NaT,NaN,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25


,sample_id,file_path,text,tabel
0,1,datasets/train_data/1.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."
1,2,datasets/train_data/2.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."


,sample_id,file_path,text,tabel
0,11188,datasets/test_data/11188.PDF,['北京京西文化旅游股份有限公司监事会\n \n \n关于使用部分闲置募集资金购买理财产品的...,[]
1,11189,datasets/test_data/11189.PDF,['北京京西文化旅游股份有限公司 \n监事会关于使用部分自有资金购买理财产品的意见 \n根据...,[]


In [4]:
# 构造训练集验证集
train_df = train_df.sample(frac=1, random_state=1017)
val_df = train_df[:1800]
train_df = train_df[1800:]

# 数据处理
## 抽取整体数据（一个sampleid内此字段内容都相同）
## 公告时间，实际购买公司

In [5]:
# 提取公司
# train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')
# result_matrix
train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')

train_lstm_input = train_lstm_input.fillna('否')

# label_1理财类型-10  label_2资金来源-3 label_3实际购买公司和上市公司关系-3 label_4买卖方是否有关联关系-2
from sklearn.preprocessing import LabelEncoder
label_1 = LabelEncoder()
# label_2 = LabelEncoder()
# label_3 = LabelEncoder()
# label_4 = LabelEncoder()

train_data = pd.DataFrame()
tmp=pd.DataFrame()
train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) 

# train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) + '_' + train_lstm_input['产品发行方名称'].astype(str)

# train_data['text_2'] = train_lstm_input['text'].astype(str)

# train_lstm_input["文本类别"]="理财产品"

train_data['label_1'] = "理财产品"


train_data2=train_lstm_input[train_lstm_input["产品发行方名称"]!="否"].reset_index(drop=True)

# train_data2["文本类别"]="发行方"

tmp['text_1']=train_data2["产品发行方名称"].astype(str)

# tmp['text_2']= train_data2["text"].astype(str)

tmp['label_1']="发行方"

train_data = pd.concat([train_data,tmp]).reset_index(drop=True)

train_data2=train_lstm_input[train_lstm_input["实际购买公司名称"]!="否"].reset_index(drop=True)

# train_data2["文本类别"]="发行方"

tmp['text_1']=train_data2["实际购买公司名称"].astype(str)

# tmp['text_2']= train_data2["text"].astype(str)

tmp['label_1']="购买公司"

train_data = pd.concat([train_data,tmp]).reset_index(drop=True)


other_columns_list=["认购金额(万元)","认购日期","资金来源","实际购买公司和上市公司关系"]

for item in other_columns_list:

    train_data2=train_lstm_input[train_lstm_input[item]!="否"].reset_index(drop=True)

    # train_data2["文本类别"]=item

    tmp['text_1']=train_data2[item].astype(str)

    # tmp['text_2']= train_data2["text"].astype(str)

    tmp['label_1']="其它"

    
    train_data = pd.concat([train_data,tmp]).reset_index(drop=True)



# train_data['label_2'] = label_2.fit_transform(train_lstm_input['资金来源'])
# train_data['label_3'] = label_3.fit_transform(train_lstm_input['实际购买公司和上市公司关系'])
# train_data['label_4'] = label_4.fit_transform(train_lstm_input['买卖方是否有关联关系'])
train_data

train_src=[]
for text,label in train_data[["text_1","label_1"]].values:
    train_src.append([label,text])


grocery_word_selector=Grocery("wordSelector")


grocery_word_selector.train(train_src)

grocery_word_selector.save()



,text_1,label_1
0,中银保本理财-人民币按期开放理财产品,理财产品
1,中银保本理财-人民币按期开放理财产品,理财产品
2,与利率挂钩的结构性产品,理财产品
3,广发银行“薪加薪”16号XJXCKJ2578,理财产品
4,兴业银行“金雪球-优悦”保本开放式人民币理财产品(2M),理财产品
...,...,...
185903,控股参股公司,其它
185904,控股参股公司,其它
185905,控股参股公司,其它
185906,控股参股公司,其它


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lsqlh\AppData\Local\Temp\jieba.cache
Loading model cost 1.063 seconds.
Prefix dict has been built successfully.


In [8]:
import jieba.analyse

text_list=[]
for product_name in tqdm(train_outputs["产品发行方名称"].values):
    text_list.append(str(product_name))

# text_list

a_list=[]

for x in jieba.analyse.extract_tags((",").join(i for i in text_list),topK=100):#可以再添加一个参数指定输出个数
    a_list.append(x)#直接输出关键词和词频

a_list

# text_list=[]
# for product_name in tqdm(train_outputs["理财产品名称"].values):
#     text_list.append(str(product_name))

# b_list=[]

# for x in jieba.analyse.extract_tags((",").join(i for i in text_list)):#可以再添加一个参数指定输出个数
#     b_list.append(x)

# set(b_list).difference(a_list)

100%|██████████| 32818/32818 [00:00<00:00, 456291.91it/s]


['支行',
 '有限公司',
 '股份',
 '分行',
 '兴业银行',
 '银行',
 '招商银行',
 '交通银行',
 '上海浦东发展银行',
 '厦门',
 '中国银行',
 '中信银行',
 '中国民生银行',
 '中国工商银行',
 '中国农业银行',
 'nan',
 '宁波',
 '台州',
 '平安',
 '上海',
 '证券',
 '浦发银行',
 '杭州',
 '广发',
 '中国光大银行',
 '民生银行',
 '农业银行',
 '文滨',
 '深圳',
 '中国建设银行',
 '信托',
 '上海银行',
 '营业部',
 '合肥',
 '北京',
 '工商银行',
 '建设银行',
 '商业银行',
 '南京',
 '张家港',
 '无锡',
 '广州',
 '北京分行',
 '国际',
 '江苏',
 '农村',
 '开发区',
 '光大银行',
 '成都',
 '福州',
 '长沙',
 '松江',
 '椒江',
 '湖州',
 '华夏银行',
 '苏州',
 '台州市',
 '有限责任',
 '管理',
 '临海',
 '东莞',
 '浙商',
 '天水',
 '集美',
 '中山',
 '中融',
 '分理处',
 '中信证券',
 '海通',
 '汕头',
 '中国',
 '闵行',
 '农商',
 '嘉善',
 '坂田',
 '淮安',
 '海沧',
 '资产',
 '农行',
 '交行',
 '青岛',
 '国泰君安证券股份有限公司',
 '哈尔滨',
 '财通',
 '巴南',
 '绍兴',
 '惠州',
 '玉环',
 '吴江',
 '临安',
 '烟台',
 '灵桥',
 '徽商',
 '银河证券',
 '天津',
 '中行',
 '民生',
 '高新',
 '建投',
 '益阳']

In [ ]:
firm_pos_word_list=['公司', '银行', '信托', '证券',  '分行', '支行', '中心', '业部', '商行', '建行']


#### 1.抽取公告时间

In [4]:
# 首先针对任务抽取时间（每个时间跟每个id是一一对应的）
# 要不是取第一个时间，要不就是取最后一个时间（或者时间加一）这里可以建立一个模型预测
# base这里面直接取最后一个时间作为发布日期

CN_NUM = {
    u'〇': 0, u'一': 1, u'二': 2, u'三': 3,
    u'四': 4, u'五': 5, u'六': 6, u'七': 7,
    u'八': 8, u'九': 9, u'零': 0, u'壹': 1,
    u'贰': 2, u'叁': 3, u'肆': 4, u'伍': 5,
    u'陆': 6, u'柒': 7, u'捌': 8, u'玖': 9,
    u'貮': 2, u'两': 2,
}


def get_put_time_from_text(row):
    row = row.replace(' ', '').replace('\\n', '')
    for key in CN_NUM:
        row = row.replace(key, str(CN_NUM[key]))   
    r = row.replace("年", "-").replace("月", "-").replace("日", " ").replace("/", "-").strip()
    regex = "(\d{4}-\d{1,2}-\d{1,2})"
    r = re.findall(regex, r)
    if len(r)==0:
        return np.nan
    time_str = r[-1]
    first = time_str.split('-')[0]
    second = time_str.split('-')[1]
    last = time_str.split('-')[-1]
    second = str.zfill(second, 2)
    last = str.zfill(last, 2)
    r = '-'.join([first, second, last])
    return r

val_result = pd.DataFrame()
val_result['sample_id'] = val_df['sample_id']
val_result['predict_time'] = val_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)
test_gg = train_outputs.groupby('sample_id').apply(lambda row:list(row['公告日期'])[0]).reset_index()
test_gg.columns = ['sample_id', 'time']
val_result = pd.merge(val_result, test_gg, on='sample_id', how='left')

# 判断验证集的准确率
np.sum(val_result['predict_time'].astype(str) == val_result['time'].astype(str))/len(val_result)

val_time = val_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)
# test_time = test_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)

100%|██████████| 1800/1800 [00:00<00:00, 4269.77it/s]


0.4583333333333333

100%|██████████| 1800/1800 [00:00<00:00, 4977.69it/s]


#### 2.抽取实际购买公司

In [5]:
# 抽取购买公司
# 前几句话出现
# 将其按照\\n 和空格切割
def get_gm(row):
    result = re.split('[\\\\n ]',row)
    for i in result:
        if '公司' in i:
            return i

val_gm = val_df.progress_apply(lambda row:get_gm(row['text']), axis=1)
# test_gm = test_df.progress_apply(lambda row:get_gm(row['text']), axis=1)

100%|██████████| 1800/1800 [00:00<00:00, 4470.96it/s]


In [ ]:
# 最后一部分字段采用预测好的部分，跟提取的text做交互采用双输入lstm在dense层做交互预测最后几个字段

# train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')
result_matrix
train_lstm_input = pd.merge(train_table_df, train_outputs, on='sample_id', how='left')

train_lstm_input = train_lstm_input.fillna('否')

# label_1理财类型-10  label_2资金来源-3 label_3实际购买公司和上市公司关系-3 label_4买卖方是否有关联关系-2
from sklearn.preprocessing import LabelEncoder
label_1 = LabelEncoder()
# label_2 = LabelEncoder()
# label_3 = LabelEncoder()
# label_4 = LabelEncoder()

train_data = pd.DataFrame()
tmp=pd.DataFrame()
train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) 

# train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) + '_' + train_lstm_input['产品发行方名称'].astype(str)

train_data['text_2'] = train_lstm_input['text'].astype(str)

train_lstm_input["文本类别"]="理财产品"

train_data['label_1'] = label_1.fit_transform(train_lstm_input["文本类别"])


train_data2=train_lstm_input[train_lstm_input["产品发行方名称"]!="无"].reset_index(drop=True)

train_data2["文本类别"]="发行方"

tmp['text_1']=train_data2["产品发行方名称"].astype(str)

tmp['text_2']= train_data2["text"].astype(str)

tmp['label_1']=label_1.fit_transform(train_data2["文本类别"])

train_data = pd.concat([train_data,tmp]).reset_index(drop=True)


other_columns_list=["认购金额(万元)","认购日期"]

for item in other_columns_list:

    train_data2=train_lstm_input[train_lstm_input[item]!="无"].reset_index(drop=True)

    train_data2["文本类别"]="其他"

    tmp['text_1']=train_data2[item].astype(str)

    tmp['text_2']= train_data2["text"].astype(str)

    tmp['label_1']=label_1.fit_transform(train_data2["文本类别"])

    
    train_data = pd.concat([train_data,tmp]).reset_index(drop=True)



# train_data['label_2'] = label_2.fit_transform(train_lstm_input['资金来源'])
# train_data['label_3'] = label_3.fit_transform(train_lstm_input['实际购买公司和上市公司关系'])
# train_data['label_4'] = label_4.fit_transform(train_lstm_input['买卖方是否有关联关系'])
train_data
